In [24]:
import tkinter as tk
from tkinter import ttk, filedialog

# Configuration de l'application principale Tkinter
root = tk.Tk()
root.title("Explorateur de Fichiers")
root.geometry("600x500")  # Taille de la fenêtre définie à 600x500

# Dictionnaire pour stocker les chemins des fichiers sélectionnés
parameters_dict = {}

# Configuration du notebook (les onglets)
notebook = ttk.Notebook(root)
notebook.pack(fill='both', expand=True)  # S'assurer que le notebook remplisse l'espace disponible

# Création des onglets avec configuration pour occuper toute la fenêtre
frame1 = ttk.Frame(notebook)
frame2 = ttk.Frame(notebook)
frame3 = ttk.Frame(notebook)
frame4 = ttk.Frame(notebook)

# S'assurer que les cadres remplissent bien l'espace
for frame in (frame1, frame2, frame3, frame4):
    frame.pack(fill='both', expand=True)  # Utiliser pack avec fill et expand pour les cadres

notebook.add(frame1, text='INPUT')
notebook.add(frame2, text='OUTPUT')
notebook.add(frame3, text='Classyfire')
notebook.add(frame4, text='NPclassifier')


# Fonction pour l'explorateur de fichiers
def browse_file(value_key, filetypes):
    file_path = filedialog.askopenfilename(filetypes=filetypes)
    if file_path:
        parameters_dict[value_key] = file_path  # Enregister le chemin dans le dictionnaire
    print(parameters_dict)


# Fonctions de gestion des changements de sélection pour les entrées de texte
def update_parameters_dict(entry, key):
    parameters_dict[key] = entry.get()
    print(parameters_dict)


# Label et bouton de l'explorateur de fichiers pour l'onglet INPUT
label_input = ttk.Label(frame1, text="Sélectionner un fichier CSV pour INPUT:")
label_input.pack(pady=10)
btn_browse_input = ttk.Button(frame1, text="Parcourir",
                              command=lambda: browse_file('input_file', [('CSV files', '*.csv')]))
btn_browse_input.pack(pady=10)

# Label et bouton de l'explorateur de fichiers pour l'onglet OUTPUT
label_output = ttk.Label(frame2, text="Sélectionner un fichier pour OUTPUT:")
label_output.pack(pady=10)
btn_browse_output = ttk.Button(frame2, text="Parcourir",
                               command=lambda: browse_file('output_file', [('All files', '*.*')]))
btn_browse_output.pack(pady=10)

# Gestionnaire pour les onglets "Classyfire"
label_classyfire = ttk.Label(frame3, text="Activates ?")
label_classyfire.pack(pady=5)
# Combobox pour ON/OFF pour Classyfire
combobox_classyfire_status = ttk.Combobox(frame3, values=["ON", "OFF"])
combobox_classyfire_status.set("ON")  # Par défaut sur "ON"
combobox_classyfire_status.pack(pady=2)
combobox_classyfire_status.bind("<<ComboboxSelected>>",
                                lambda event: update_parameters_dict(combobox_classyfire_status, 'classyfire_status'))

# Gestionnaire pour les onglets "Classyfire"
label_classyfire = ttk.Label(frame3, text="inchikeys column:")
label_classyfire.pack(pady=5)
# Entry pour saisir directement la colonne inchikeys dans l'onglet Classyfire
entry_classyfire_column = ttk.Entry(frame3)
entry_classyfire_column.pack(pady=10)
entry_classyfire_column.bind("<KeyRelease>",
                             lambda event: update_parameters_dict(entry_classyfire_column, 'classyfire_column'))

# Gestionnaire pour les onglets "NPclassifier"
label_npclassifier = ttk.Label(frame4, text="Activate ?")
label_npclassifier.pack(pady=5)
# Combobox pour ON/OFF pour NPclassifier
combobox_npclassifier_status = ttk.Combobox(frame4, values=["ON", "OFF"])
combobox_npclassifier_status.set("ON")  # Par défaut sur "ON"
combobox_npclassifier_status.pack(pady=2)
combobox_npclassifier_status.bind("<<ComboboxSelected>>",
                                  lambda event: update_parameters_dict(combobox_npclassifier_status,
                                                                       'npclassifier_status'))

# Gestionnaire pour les onglets "NPclassifier"
label_npclassifier = ttk.Label(frame4, text="smiles column:")
label_npclassifier.pack(pady=5)
# Entry pour saisir directement la colonne smiles dans l'onglet NPclassifier
entry_npclassifier_column = ttk.Entry(frame4)
entry_npclassifier_column.pack(pady=10)
entry_npclassifier_column.bind("<KeyRelease>",
                               lambda event: update_parameters_dict(entry_npclassifier_column, 'npclassifier_column'))

root.mainloop()


{}
